## Satisfaction Analysis - Task 4

In [1]:
# Import packages
import os,sys

# Add the path to the directory
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)
    
from scripts.utils import *
from db_connection.postgresql_connection import *

In [2]:
# Load dataset from postgres
telecom_df = fetch_data_from_postgres()


In [3]:
# Dataframe head
telecom_df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [4]:
# filling missing value with mean or mode
fill_missing_values_t3(telecom_df)


In [5]:
# process df
telecom_df['Total_Avg_RTT'] = telecom_df['Avg RTT DL (ms)'] + telecom_df['Avg RTT UL (ms)']
telecom_df['Total_Avg_Bearer_TP'] = telecom_df['Avg Bearer TP DL (kbps)'] + telecom_df['Avg Bearer TP UL (kbps)']
telecom_df['Total_Avg_TCP'] = telecom_df['TCP DL Retrans. Vol (Bytes)'] + telecom_df['TCP UL Retrans. Vol (Bytes)']
telecom_df['Total TCP Retrans. Vol (Bytes)'] = telecom_df['TCP DL Retrans. Vol (Bytes)'] + telecom_df['TCP UL Retrans. Vol (Bytes)']

# user experience scaled
user_experience = telecom_df.set_index('MSISDN/Number')[
     ['Total_Avg_RTT', 'Total_Avg_Bearer_TP', 'Total TCP Retrans. Vol (Bytes)']]
scaler = StandardScaler()
scaled_array = scaler.fit_transform(user_experience)
pd.DataFrame(scaled_array).head(5)


,0,1,2
0,-0.142369,-0.559296,3.145329e-17
1,-0.101671,-0.560227,3.145329e-17
2,0.000000,-0.561234,3.145329e-17
3,0.000000,-0.558513,3.145329e-17
4,0.000000,-0.561234,3.145329e-17


In [6]:
user_experience

,Total_Avg_RTT,Total_Avg_Bearer_TP,Total TCP Retrans. Vol (Bytes)
MSISDN/Number,,,
3.366496e+10,47.000000,67.000000,2.156957e+07
3.368185e+10,70.000000,42.000000,2.156957e+07
3.376063e+10,127.458589,15.000000,2.156957e+07
3.375034e+10,127.458589,88.000000,2.156957e+07
3.369980e+10,127.458589,15.000000,2.156957e+07
...,...,...,...
3.365069e+10,32.000000,117.000000,2.156957e+07
3.366345e+10,29.000000,77.000000,2.156957e+07
3.362189e+10,49.000000,90.000000,2.156957e+07


In [7]:
# load user engagement 

user_engagement = pd.read_csv("../data/user_engagement.csv")

Task 4.1

Engagement score calculated and assigned


Assign experience score to each user

In [8]:
# scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(user_engagement)
scaled_data

array([[-3.99191484e-03, -2.08088240e-01, -4.89228047e-01, ...,
         3.80796440e-01,  1.55416341e+00,  1.55416341e+00],
       [-3.99191483e-03,  9.93460814e-01, -4.89228047e-01, ...,
        -1.08775937e+00, -2.62956868e-01, -2.62956868e-01],
       [-3.99191457e-03,  9.93460814e-01, -4.89228047e-01, ...,
        -1.94402111e-01, -2.62956868e-01, -2.62956868e-01],
       ...,
       [ 1.08886651e+00, -1.40963729e+00, -4.89228047e-01, ...,
        -9.34635054e-01,  6.45603272e-01,  6.45603272e-01],
       [ 1.16406925e+02,  9.93460814e-01, -4.89228047e-01, ...,
        -1.93748347e-01,  6.45603272e-01,  6.45603272e-01],
       [ 3.04823830e+02, -1.40963729e+00, -4.89228047e-01, ...,
        -1.12378199e+00,  6.45603272e-01,  6.45603272e-01]])

In [9]:
# normalize data

normalized_data = normalize(scaled_data)
normalized_data

array([[-0.00173658, -0.0905235 , -0.21282622, ...,  0.1656558 ,
         0.67609926,  0.67609926],
       [-0.00248413,  0.61822022, -0.30444147, ..., -0.67690122,
        -0.16363529, -0.16363529],
       [-0.00336625,  0.83775169, -0.41254936, ..., -0.16393268,
        -0.22174258, -0.22174258],
       ...,
       [ 0.24070047, -0.31160877, -0.10814679, ..., -0.20660668,
         0.14271447,  0.14271447],
       [ 0.9999106 ,  0.00853362, -0.00420236, ..., -0.00166426,
         0.00554559,  0.00554559],
       [ 0.99989588, -0.00462395, -0.00160479, ..., -0.00368628,
         0.00211773,  0.00211773]])

Engagement_score

In [ ]:
# Load K-means model from a pickle file
with open("../models/user_engagement_cluster.pkl", "rb") as f:
    k_means1 = pickle.load(f)

In [11]:
less_engaged_cluster = 3

distance = k_means1.fit_transform(normalized_data)
distance_from_less_engaged_cluster = list(
    map(lambda x: x[less_engaged_cluster], distance))
user_engagement['engagement_score'] = distance_from_less_engaged_cluster
user_engagement

,MSISDN/Number,Cluster,sessions,time_duration,Total_UL_and_DL,cluster,best_cluster,engagement_score
0,3.360100e+10,1,1,116720.0,8.786906e+08,3,3,0.971212
1,3.360100e+10,2,1,181230.0,1.568596e+08,1,1,0.920024
2,3.360100e+10,2,1,134969.0,5.959665e+08,1,1,0.979485
3,3.360101e+10,2,1,49878.0,4.223207e+08,1,1,1.082198
4,3.360101e+10,0,2,37104.0,1.457411e+09,0,0,1.924275
...,...,...,...,...,...,...,...,...
106466,3.379000e+10,1,1,8810.0,7.146416e+08,3,3,1.026010
106467,3.379000e+10,2,1,140988.0,4.803073e+08,1,1,0.947296
106468,3.197021e+12,0,1,877385.0,2.321240e+08,2,2,1.173454
106469,3.370000e+14,2,1,253030.0,5.962878e+08,2,2,1.374102


Experience score

In [ ]:
# Load K-means model from a pickle file
with open("../models/user_exp_cluster.pkl", "rb") as f:
    k_means2 = pickle.load(f)

In [13]:
# scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(user_experience)
scaled_data

array([[-1.42368580e-01, -5.59295518e-01,  3.14532900e-17],
       [-1.01670907e-01, -5.60227461e-01,  3.14532900e-17],
       [ 0.00000000e+00, -5.61233960e-01,  3.14532900e-17],
       ...,
       [-1.38829652e-01, -5.58438130e-01,  3.14532900e-17],
       [-1.51215900e-01, -5.59146407e-01,  3.14532900e-17],
       [ 0.00000000e+00,  6.78078011e-17,  3.14532900e-17]])

In [14]:
# normalize data
normalized_data = normalize(scaled_data)
normalized_data

array([[-2.46683264e-01, -9.69096160e-01,  5.44993864e-17],
       [-1.78564736e-01, -9.83928166e-01,  5.52414512e-17],
       [ 0.00000000e+00, -1.00000000e+00,  5.60430983e-17],
       ...,
       [-2.41259852e-01, -9.70460552e-01,  5.46599087e-17],
       [-2.61062269e-01, -9.65321963e-01,  5.43016127e-17],
       [ 0.00000000e+00,  6.78078011e-17,  3.14532900e-17]])

In [15]:
less_experience_cluster = 1

distance = k_means2.fit_transform(normalized_data)
distance_from_less_experience_cluster = list(
    map(lambda x: x[less_experience_cluster], distance))
user_experience['experience_score'] = distance_from_less_experience_cluster
user_experience

,Total_Avg_RTT,Total_Avg_Bearer_TP,Total TCP Retrans. Vol (Bytes),experience_score
MSISDN/Number,,,,
3.366496e+10,47.000000,67.000000,2.156957e+07,1.878955
3.368185e+10,70.000000,42.000000,2.156957e+07,1.888855
3.376063e+10,127.458589,15.000000,2.156957e+07,1.903883
3.375034e+10,127.458589,88.000000,2.156957e+07,1.903883
3.369980e+10,127.458589,15.000000,2.156957e+07,1.903883
...,...,...,...,...
3.365069e+10,32.000000,117.000000,2.156957e+07,1.871374
3.366345e+10,29.000000,77.000000,2.156957e+07,1.869938
3.362189e+10,49.000000,90.000000,2.156957e+07,1.879833


Task 4.2

Top 10 satisfied customers based on the average of engagement & experience scores

In [16]:
# get the user satisfaction data frame
# rename column names for the satisfaction score dataframe merger
user_engagement.rename(columns={'cluster': 'engagement_cluster'}, inplace=True)
user_satisfaction_df = pd.merge(user_engagement, user_experience, on='MSISDN/Number')
user_satisfaction_df

,MSISDN/Number,Cluster,sessions,time_duration,Total_UL_and_DL,engagement_cluster,best_cluster,engagement_score,Total_Avg_RTT,Total_Avg_Bearer_TP,Total TCP Retrans. Vol (Bytes),experience_score
0,3.360100e+10,1,1,116720.0,8.786906e+08,3,3,0.971212,46.000000,76.0,2.156957e+07,1.878460
1,3.360100e+10,2,1,181230.0,1.568596e+08,1,1,0.920024,31.000000,99.0,2.156957e+07,1.870913
2,3.360100e+10,2,1,134969.0,5.959665e+08,1,1,0.979485,127.458589,97.0,2.156957e+07,1.903883
3,3.360101e+10,2,1,49878.0,4.223207e+08,1,1,1.082198,84.000000,248.0,7.607247e+05,1.843662
4,3.360101e+10,0,2,37104.0,1.457411e+09,0,0,1.924275,91.000000,56781.0,9.370832e+06,0.155605
...,...,...,...,...,...,...,...,...,...,...,...,...
148544,3.379000e+10,1,1,8810.0,7.146416e+08,3,3,1.026010,52.000000,10365.0,2.180450e+05,1.624951
148545,3.379000e+10,2,1,140988.0,4.803073e+08,1,1,0.947296,40.000000,116.0,2.156957e+07,1.875442
148546,3.197021e+12,0,1,877385.0,2.321240e+08,2,2,1.173454,127.458589,1.0,2.156957e+07,1.903883
148547,3.370000e+14,2,1,253030.0,5.962878e+08,2,2,1.374102,127.458589,33.0,2.156957e+07,1.903883


In [17]:
# get the satisfaction score based on the average of the engagement and experience scores
user_satisfaction_df['satisfaction_score'] = (
    user_satisfaction_df['engagement_score'] + user_satisfaction_df['experience_score'])/2
user_satisfaction_df

,MSISDN/Number,Cluster,sessions,time_duration,Total_UL_and_DL,engagement_cluster,best_cluster,engagement_score,Total_Avg_RTT,Total_Avg_Bearer_TP,Total TCP Retrans. Vol (Bytes),experience_score,satisfaction_score
0,3.360100e+10,1,1,116720.0,8.786906e+08,3,3,0.971212,46.000000,76.0,2.156957e+07,1.878460,1.424836
1,3.360100e+10,2,1,181230.0,1.568596e+08,1,1,0.920024,31.000000,99.0,2.156957e+07,1.870913,1.395469
2,3.360100e+10,2,1,134969.0,5.959665e+08,1,1,0.979485,127.458589,97.0,2.156957e+07,1.903883,1.441684
3,3.360101e+10,2,1,49878.0,4.223207e+08,1,1,1.082198,84.000000,248.0,7.607247e+05,1.843662,1.462930
4,3.360101e+10,0,2,37104.0,1.457411e+09,0,0,1.924275,91.000000,56781.0,9.370832e+06,0.155605,1.039940
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148544,3.379000e+10,1,1,8810.0,7.146416e+08,3,3,1.026010,52.000000,10365.0,2.180450e+05,1.624951,1.325480
148545,3.379000e+10,2,1,140988.0,4.803073e+08,1,1,0.947296,40.000000,116.0,2.156957e+07,1.875442,1.411369
148546,3.197021e+12,0,1,877385.0,2.321240e+08,2,2,1.173454,127.458589,1.0,2.156957e+07,1.903883,1.538668
148547,3.370000e+14,2,1,253030.0,5.962878e+08,2,2,1.374102,127.458589,33.0,2.156957e+07,1.903883,1.638992


Top 10 satisfied customers

In [18]:
# Top10 satisfied users
user_satisfaction_df_ = user_satisfaction_df[['MSISDN/Number', 'engagement_score', 'experience_score', 'satisfaction_score']]
user_satisfaction_df_.set_index('MSISDN/Number', inplace=True)

top10_satisfied = user_satisfaction_df_.sort_values('satisfaction_score', ascending=False).head(10)
top10_satisfied

,engagement_score,experience_score,satisfaction_score
MSISDN/Number,,,
3.366507e+10,1.950169,1.903883,1.927026
3.366488e+10,1.949901,1.903883,1.926892
3.378274e+10,1.949755,1.903883,1.926819
3.369822e+10,1.949738,1.903883,1.926810
3.369822e+10,1.949738,1.903883,1.926810
3.365923e+10,1.949691,1.903883,1.926787
3.361245e+10,1.949633,1.903883,1.926758
3.366470e+10,1.949534,1.903883,1.926708
3.366199e+10,1.949466,1.903883,1.926674


Task 4.3

Regression model

In [19]:
# splitting training and testing data
X = user_satisfaction_df_[['engagement_score', 'experience_score']]
y = user_satisfaction_df_[['satisfaction_score']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [20]:
# fit a simple linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

Task 4.4

Run a k-means (k=2) on the engagement & the experience score.

In [22]:
# copy the data frame for k means clustering
Kmen_df = user_satisfaction_df_.copy(deep=True)

In [24]:
# scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(Kmen_df)
scaled_data

array([[-0.86656182,  0.66252029,  0.0977948 ],
       [-0.97710949,  0.65206144,  0.02305933],
       [-0.84869515,  0.69774785,  0.1406686 ],
       ...,
       [-0.42979113,  0.69774785,  0.38747581],
       [ 0.00353668,  0.69774785,  0.64278114],
       [ 0.01695123,  0.69774785,  0.65068464]])

In [25]:
# normalize data
normalized_data = normalize(scaled_data)
normalized_data

array([[-0.79124824,  0.60494012,  0.08929538],
       [-0.8316326 ,  0.55497931,  0.01962615],
       [-0.76620157,  0.62992642,  0.12699554],
       ...,
       [-0.4741308 ,  0.76973144,  0.42744999],
       [ 0.00372792,  0.73547767,  0.67753871],
       [ 0.01776455,  0.73122562,  0.68190433]])

In [26]:
# get a k means cluster of 2 clusters
kmeans = KMeans(n_clusters=2, random_state=1).fit(normalized_data)
kmeans.labels_

array([1, 1, 1, ..., 1, 1, 1], dtype=int32)

In [27]:
# insert the clusters into the data frame
user_satisfaction_df_.insert(0, 'satisfaction_cluster', kmeans.labels_)
user_satisfaction_df_

,satisfaction_cluster,engagement_score,experience_score,satisfaction_score
MSISDN/Number,,,,
3.360100e+10,1,0.971212,1.878460,1.424836
3.360100e+10,1,0.920024,1.870913,1.395469
3.360100e+10,1,0.979485,1.903883,1.441684
3.360101e+10,1,1.082198,1.843662,1.462930
3.360101e+10,0,1.924275,0.155605,1.039940
...,...,...,...,...
3.379000e+10,1,1.026010,1.624951,1.325480
3.379000e+10,1,0.947296,1.875442,1.411369
3.197021e+12,1,1.173454,1.903883,1.538668


In [28]:
# take a look at the number of users in each group
user_satisfaction_df_['satisfaction_cluster'].value_counts()

satisfaction_cluster
1    104845
0     43704
Name: count, dtype: int64

Task 4.5

Aggregate the average satisfaction & experience score per cluster.

In [34]:
# average satisfaction & experience score per cluster
average_satisfaction = user_satisfaction_df_.groupby('satisfaction_cluster').agg(
    {'satisfaction_score': 'mean', 'experience_score': 'mean', 'engagement_score' : 'mean'})

average_satisfaction

,satisfaction_score,experience_score,engagement_score
satisfaction_cluster,,,
0,0.905580,0.320362,1.490799
1,1.586837,1.850536,1.323138


Task 4.6

Export your final table containing all user ID + engagement, experience &
satisfaction scores

In [52]:
# write df to the pstgres
write_dataframe_to_postgres(user_satisfaction_df_,'user_satisfaction_df')

DataFrame successfully written to 'user_satisfaction_df' table.


In [45]:
user_satisfaction_df_

,MSISDN/Number,satisfaction_cluster,engagement_score,experience_score,satisfaction_score
0,3.360100e+10,1,0.971212,1.878460,1.424836
1,3.360100e+10,1,0.920024,1.870913,1.395469
2,3.360100e+10,1,0.979485,1.903883,1.441684
3,3.360101e+10,1,1.082198,1.843662,1.462930
4,3.360101e+10,0,1.924275,0.155605,1.039940
...,...,...,...,...,...
148544,3.379000e+10,1,1.026010,1.624951,1.325480
148545,3.379000e+10,1,0.947296,1.875442,1.411369
148546,3.197021e+12,1,1.173454,1.903883,1.538668
148547,3.370000e+14,1,1.374102,1.903883,1.638992


In [ ]:
# write datframe to postgres
write_dataframe_to_postgres(user_satisfaction_df_,'user_satisfaction_df')

In [50]:
# featch from postgresql
df = fetch_data_from_postgres('user_satisfaction_df')


,MSISDN/Number,satisfaction_cluster,engagement_score,experience_score,satisfaction_score
0,33601001722,1,0.971212,1.878460,1.424836
1,33601001754,1,0.920024,1.870913,1.395469
2,33601002511,1,0.979485,1.903883,1.441684
3,33601007832,1,1.082198,1.843662,1.462930
4,33601008617,0,1.924275,0.155605,1.039940
...,...,...,...,...,...
148544,33789996170,1,1.026010,1.624951,1.325480
148545,33789997247,1,0.947296,1.875442,1.411369
148546,3197020876596,1,1.173454,1.903883,1.538668
148547,337000037000919,1,1.374102,1.903883,1.638992
